In [1]:
from fixed_income import market_data as md
from fixed_income import instruments
from fixed_income.dates import *

# Problem 6 & 7

In [2]:
file_path = rf"C:\Users\anjal\Desktop\Fordham\Spring 2026\Interest Rate Derivatives\Homework\fixed_income\data\hw3_data\hw3_new_data.xlsx"
df = md.read_excel_bbg_fff(file_path)

df = df[df["open_interest"] > 2000]

In [3]:
df

,ticker,close_price,open_interest
0,FFF6,96.360,772515
1,FFG6,96.415,676065
2,FFH6,96.450,194654
3,FFJ6,96.515,178202
4,FFK6,96.575,171275
5,FFN6,96.715,66204
6,FFM6,96.635,50438
7,FFQ6,96.785,46536
8,FFV6,96.880,24631
9,FFX6,96.915,23648


In [4]:
import pandas as pd

contracts = [
    instruments.FedFundsFutures(t, p)
    for t, p in zip(df["ticker"], df["close_price"])
]

In [5]:
c = contracts[0]
print(c.ticker, c.start_date, c.end_date, c.price, c.implied_rate)

FFF6 2026-01-01 2026-01-31 96.36 3.64


In [6]:
# Read the EFFR on 2025-12-17
effr_20251217 = pd.read_excel(r"C:\Users\anjal\Desktop\Fordham\Spring 2026\Interest Rate Derivatives\Homework\fixed_income\data\hw3_data\effr_20251217.xlsx")
effr_20251217
effr = effr_20251217.loc[0, "effr"]
print(f"EFFR on 2025-12-17: {effr}%")

EFFR on 2025-12-17: 3.64%


In [7]:
df

,ticker,close_price,open_interest
0,FFF6,96.360,772515
1,FFG6,96.415,676065
2,FFH6,96.450,194654
3,FFJ6,96.515,178202
4,FFK6,96.575,171275
5,FFN6,96.715,66204
6,FFM6,96.635,50438
7,FFQ6,96.785,46536
8,FFV6,96.880,24631
9,FFX6,96.915,23648


# Problem 8 
Function implementation in fixed_income/dates/utils.py:  
```def split_month_by_date(month_begin: dt.date, month_end: dt.date, split_date: dt.date) -> tuple[float, float]:```


# Problem 9
Implementation in instruments/fed_funds_futures.py

In [8]:
from fixed_income.dates.calendars import get_fomc_meeting_dates

# Ensure contracts are sorted
contracts = sorted(contracts, key=lambda c: c.start_date)

start_year = contracts[0].start_date.year
end_year = contracts[-1].end_date.year

fomc_dates = []
for y in range(start_year, end_year + 1):
    fomc_dates += get_fomc_meeting_dates(y)

# keep only meetings inside the futures horizon
fomc_dates = [d for d in fomc_dates if contracts[0].start_date <= d <= contracts[-1].end_date]
fomc_dates = sorted(fomc_dates)

fomc_dates

[datetime.date(2026, 1, 28),
 datetime.date(2026, 3, 18),
 datetime.date(2026, 4, 29),
 datetime.date(2026, 6, 17),
 datetime.date(2026, 7, 29),
 datetime.date(2026, 9, 16),
 datetime.date(2026, 10, 28),
 datetime.date(2026, 12, 9),
 datetime.date(2027, 1, 27)]

### (1) via bootstrapping

In [9]:
expect = instruments.FedFundsFutures.extract_fomc_expectations_bootstrap(
    contracts=contracts,
    effr=effr,
    fomc_dates=fomc_dates
)


for d, r in sorted(expect.items()):
    print(d, r)

2026-01-28 3.585
2026-03-18 3.507
2026-04-29 3.425
2026-06-17 3.296
2026-07-29 3.215
2026-09-16 3.135
2026-10-28 3.085
2026-12-09 3.045
2027-01-27 3.05


### (2) via least squares

In [10]:
lsq_expect = instruments.FedFundsFutures.extract_fomc_expectations_lstsq(
    contracts=contracts,
    effr=3.64,
    fomc_dates=fomc_dates
)

for d, r in sorted(lsq_expect.items()):
    print(d, r)

2026-01-28 3.589
2026-03-18 3.492
2026-04-29 3.426
2026-06-17 3.293
2026-07-29 3.216
2026-09-16 3.127
2026-10-28 3.085
2026-12-09 3.044
2027-01-27 3.05


# Problem 10

In [11]:
from fixed_income.curves import FFERCurve
import datetime as dt

curve_date = dt.date(2025, 12, 17)

meeting_calendar = []
for y in [2025, 2026, 2027]:
    meeting_calendar += get_fomc_meeting_dates(y)
meeting_calendar = sorted(meeting_calendar)

curve = FFERCurve(curve_date, effr, lsq_expect, meeting_calendar)

df_to_feb1 = curve.discount_factor(dt.date(2026, 2, 1))
df_to_cutoff = curve.discount_factor(curve.max_supported_date())

df_to_feb1, curve.max_supported_date(), df_to_cutoff

fwd = curve.implied_simple_forward_rate(dt.date(2026,2,1), curve.max_supported_date())

In [12]:
print(f"Discount factor to 2026-02-01: {df_to_feb1:.6f}")
print(f"Discount factor to curve cutoff ({curve.max_supported_date()}): {df_to_cutoff:.6f}")
print(f"Forward rate (ACT/360) from 2026-02-01 to cutoff: {fwd*100:.2f}%")

Discount factor to 2026-02-01: 0.995367
Discount factor to curve cutoff (2027-03-17): 0.959242
Forward rate (ACT/360) from 2026-02-01 to cutoff: 3.31%


# Problem 11

In [13]:
fwd_date = dt.date(2026, 1, 2)
mat_date = dt.date(2026, 7, 1)

df_fwd = curve.discount_factor_fwd(mat_date, fwd_date)
y_fwd = curve.zcb_yield(mat_date, fwd_date)
mm = curve.money_market_rate(fwd_date, mat_date)  # spot by default (forward_curve_date = curve_date)

print(f"Forward DF: {df_fwd:.3f}")
print(f"Forward ZCB yield: {y_fwd * 100:.2f}%")
print(f"MM term rate: {mm * 100:.2f}%")

Forward DF: 0.983
Forward ZCB yield: 3.50%
MM term rate: 3.54%


In [14]:
d = dt.date(2026, 2, 1)
f1d = curve.daily_forward_rate(d)
print(f"1-day forward rate on {d} = {f1d*100:.2f}%")

1-day forward rate on 2026-02-01 = 3.59%


# Problem 12

In [15]:
import datetime as dt

d_target = dt.date(2026, 8, 6)

d_fwd_start = dt.date(2026, 6, 6)
d_fwd_end   = dt.date(2026, 9, 6)

# (1) DF(8/6/2026)
df_target = curve.discount_factor(d_target)

# (2) ZCB yield (continuous compounding) for 8/6/2026
zcb_yield_target = curve.zcb_yield(d_target)   # returns decimal

# (3) Forward money market rate for 6/6/2026 -> 9/6/2026
# Using forward_curve_date = 6/6/2026 makes it a forward rate starting on that date.
mm_fwd = curve.money_market_rate(d_fwd_start, d_fwd_end, forward_curve_date=d_fwd_start)

print(f"Curve date: {curve.curve_date}")
print(f"DF({d_target}): {df_target:.6f}")
print(f"ZCB yield (cont) to {d_target}: {zcb_yield_target*100:.4f}%")
print(f"Forward money market rate {d_fwd_start} -> {d_fwd_end}: {mm_fwd*100:.4f}%")

Curve date: 2025-12-17
DF(2026-08-06): 0.977837
ZCB yield (cont) to 2026-08-06: 3.4778%
Forward money market rate 2026-06-06 -> 2026-09-06: 3.2876%
